In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import pandas as pd
from py_lib.dumb_extract_cat_features import get_dumb_feature_hash

wpns = pd.read_csv('data/wpns.csv')
wpns

items_categories = []
for ind, row in wpns[['type', 'translation']].iterrows():
    items_categories.append([row['type'], row['translation']])
items_categories

def get_data():
    return [
        {
            "id": 1,
            "sql_query": "select market_hash_name, sum(price)/sum(quantity) as mean_price from extended_steam_history_sellings where category = '{category}' group by market_hash_name order by mean_price desc limit {limit}",
            "templates": [
                "топ/рейтинг/покажи/_ {limit} самых/наиболее/_ дорогих скинов_в_категории_{category}/предметов_в_категории_{category}/{category} за_все_время/_",
                "какие скины_в_категории_{category}/предметы_в_категории_{category}/{category} были наиболее/самыми дорогими за_все_время/_"
            ],
            "possible_features_values": {
                "limit": [*range(1, 200, 15)],
                "category": items_categories
            }
        },
        {
            "id": 2,
            "sql_query": "select * from extended_steam_history_sellings where market_hash_name = '{market_hash_name}' limit 5000",
            "templates": [
                "открой/покажи/_ график_цены/график/динамика_цен/динамика скина/предмета/_ {market_hash_name}"
            ],
            "possible_features_values": {
                "market_hash_name": [get_dumb_feature_hash('market_hash_name')]
            },
        },
        {
            "id": 3,
            "sql_query": "select * from extended_steam_history_sellings where market_hash_name = '{market_hash_name}', dateDiff('day', timestamp, now()) < {number_of_days} limit 5000",
            "templates": [
                "открой/покажи/_ график_цены/график/динамика_цен/динамика скина/предмета/_ {market_hash_name} за_последние_{number_of_days}_дней/за_{number_of_days}_дней/за_{number_of_days}_д"
            ],
            "possible_features_values": {
                "market_hash_name": [get_dumb_feature_hash('market_hash_name')],
                "number_of_days": [*range(1, 50, 10)]
            }
        },
        {
            "id": 4,
            "sql_query": "select * from extended_steam_history_sellings where market_hash_name = '{market_hash_name}', dateDiff('week', timestamp, now()) < {number_of_weeks} limit 5000",
            "templates": [
                "открой/покажи/_ график_цены/график/динамика_цен/динамика скина/предмета/_ {market_hash_name} за_последние_{number_of_weeks}_недель/за_{number_of_weeks}_недель/за_{number_of_weeks}_н"
            ],
            "possible_features_values": {
                "market_hash_name": [get_dumb_feature_hash('market_hash_name')],
                "number_of_weeks": [*range(1, 10, 4)]
            }
        },
        {
            "id": 5,
            "sql_query": "select * from extended_steam_history_sellings where market_hash_name = '{market_hash_name}', dateDiff('month', timestamp, now()) < {number_of_months} limit 5000",
            "templates": [
                "открой/покажи/_ график_цены/график/динамика_цен/динамика скина/предмета/_ {market_hash_name} за_последние_{number_of_months}_месяцев/за_{number_of_months}_месяцев/за_{number_of_months}_м"
            ],
            "possible_features_values": {
                "market_hash_name": [get_dumb_feature_hash('market_hash_name')],
                "number_of_months": [*range(1, 15, 4)]
            }
        },

    ]

In [9]:
from py_lib.model import Text2SQLDataset, Text2SQLModel

dataset = Text2SQLDataset.generate_by_templates(get_data())
dataset.df.head()

# with open('test_out.txt', 'w') as f:
#     for ind, row in dataset.df[['query', 'sql']].iterrows():
#         f.write(str(row[0]) + '\t' + str(row[1]) + '\n')

,intent,query,query_tokens,pretty,train_data,sql,features_values
0,1,топ 1 самых дорогих скинов в категории shotgun...,"[O, B-LIMIT, O, O, O, O, O, B-CATEGORY, O, O, O]",топ 1 (B-LIMIT) самых дорогих скинов в категор...,топ O\n1 B-LIMIT\nсамых O\nдорогих O\nскинов O...,"select market_hash_name, sum(price)/sum(quanti...","{'limit': 1, 'category': 'Shotgun'}"
1,1,топ 1 самых дорогих скинов в категории shotgun,"[O, B-LIMIT, O, O, O, O, O, B-CATEGORY]",топ 1 (B-LIMIT) самых дорогих скинов в категор...,топ O\n1 B-LIMIT\nсамых O\nдорогих O\nскинов O...,"select market_hash_name, sum(price)/sum(quanti...","{'limit': 1, 'category': 'Shotgun'}"
2,1,топ 1 самых дорогих предметов в категории shot...,"[O, B-LIMIT, O, O, O, O, O, B-CATEGORY, O, O, O]",топ 1 (B-LIMIT) самых дорогих предметов в кате...,топ O\n1 B-LIMIT\nсамых O\nдорогих O\nпредмето...,"select market_hash_name, sum(price)/sum(quanti...","{'limit': 1, 'category': 'Shotgun'}"
3,1,топ 1 самых дорогих предметов в категории shotgun,"[O, B-LIMIT, O, O, O, O, O, B-CATEGORY]",топ 1 (B-LIMIT) самых дорогих предметов в кате...,топ O\n1 B-LIMIT\nсамых O\nдорогих O\nпредмето...,"select market_hash_name, sum(price)/sum(quanti...","{'limit': 1, 'category': 'Shotgun'}"
4,1,топ 1 самых дорогих shotgun за все время,"[O, B-LIMIT, O, O, B-CATEGORY, O, O, O]",топ 1 (B-LIMIT) самых дорогих shotgun (B-CATEG...,топ O\n1 B-LIMIT\nсамых O\nдорогих O\nshotgun ...,"select market_hash_name, sum(price)/sum(quanti...","{'limit': 1, 'category': 'Shotgun'}"


In [10]:
train_dataset, test_dataset, val_dataset = dataset.train_test_val_split()
train_dataset.df.shape, test_dataset.df.shape, val_dataset.df.shape

((14236, 7), (1780, 7), (1780, 7))

In [11]:
import pandas as pd
market_hash_names = pd.read_csv('data/market_hash_names.csv')
market_hash_names = list(market_hash_names['market_hash_name'].unique())
model = Text2SQLModel(dumb_features_dict={ 'market_hash_name': market_hash_names })
model.fit(train_dataset, val_dataset)

4	best: 0.0000154 (791)	total: 20.5s	remaining: 5.38s
792:	learn: 0.0000563	test: 0.0000154	best: 0.0000154 (792)	total: 20.5s	remaining: 5.35s
793:	learn: 0.0000562	test: 0.0000153	best: 0.0000153 (793)	total: 20.5s	remaining: 5.33s
794:	learn: 0.0000561	test: 0.0000153	best: 0.0000153 (794)	total: 20.6s	remaining: 5.3s
795:	learn: 0.0000560	test: 0.0000153	best: 0.0000153 (795)	total: 20.6s	remaining: 5.28s
796:	learn: 0.0000560	test: 0.0000153	best: 0.0000153 (796)	total: 20.6s	remaining: 5.25s
797:	learn: 0.0000559	test: 0.0000152	best: 0.0000152 (797)	total: 20.6s	remaining: 5.22s
798:	learn: 0.0000558	test: 0.0000152	best: 0.0000152 (798)	total: 20.7s	remaining: 5.2s
799:	learn: 0.0000557	test: 0.0000152	best: 0.0000152 (799)	total: 20.7s	remaining: 5.17s
800:	learn: 0.0000557	test: 0.0000152	best: 0.0000152 (800)	total: 20.7s	remaining: 5.14s
801:	learn: 0.0000556	test: 0.0000151	best: 0.0000151 (801)	total: 20.7s	remaining: 5.12s
802:	learn: 0.0000555	test: 0.0000151	best: 0.00

In [12]:
# model.get_intent_classifier_accuracy(test_dataset)
n = 1020
# print('query:', test_dataset.get_preprocessed_queries()[n:n+1])
# print('result', model.predict_intents_by_preprocessed_queries(test_dataset.get_preprocessed_queries()[n:n+1]))
model.evaluate([
    'динамика цены предмета ak-47 redline well-worn за последние 5 дней',
    'покажи график awp asiimov field tested за последние 10 месяцев',
    'топ 5 предметов в категории Knife',
    'какие предметы в категории machinegun были наиболее дорогими'
])

['динамика цены предмета e10793472c8aad74e4d4458b16f3e45234a1733e08dc7130bfe168479d68ddf4 за последние 5 дней', 'покажи график e10793472c8aad74e4d4458b16f3e45234a1733e08dc7130bfe168479d68ddf4 за последние 10 месяцев', 'топ 5 предметов в категории knife', 'какие предметы в категории machinegun были наиболее дорогими']


[{'raw_query': 'динамика цены предмета ak-47 redline well-worn за последние 5 дней',
  'preprocessed_query': 'динамика цены предмета e10793472c8aad74e4d4458b16f3e45234a1733e08dc7130bfe168479d68ddf4 за последние 5 дней',
  'sql': "select * from extended_steam_history_sellings where market_hash_name = 'AK-47 | Redline (Well-Worn)', dateDiff('month', timestamp, now()) < 1 limit 5000",
  'intent_id': 3},
 {'raw_query': 'покажи график awp asiimov field tested за последние 10 месяцев',
  'preprocessed_query': 'покажи график e10793472c8aad74e4d4458b16f3e45234a1733e08dc7130bfe168479d68ddf4 за последние 10 месяцев',
  'sql': "select * from extended_steam_history_sellings where market_hash_name = 'AWP | Asiimov (Field-Tested)', dateDiff('month', timestamp, now()) < 1 limit 5000",
  'intent_id': 5},
 {'raw_query': 'топ 5 предметов в категории Knife',
  'preprocessed_query': 'топ 5 предметов в категории knife',
  'sql': "select market_hash_name, sum(price)/sum(quantity) as mean_price from extended